In [220]:
import Bio.PDB
import Bio.pairwise2
import re
import glob
import pandas as pd

In [221]:
#dbfiles = str(subprocess.check_output(["find", "../data/mhc_alleles" ,"-type", "f", "-name", "*prot.fasta"]))

dbfiles = glob.glob("../data/mhc_alleles/hla/*prot.fasta") + glob.glob("../data/mhc_alleles/mhc/*/*prot.fasta")
mhc_db_path = '../sandbox/db.fasta'
epitope_db_path = '../data/epitopes/epitope_table.csv'
blast_output_path = '../sandbox/blast_output'
db = open(mhc_db_path, 'w')
for dbfile in dbfiles:
    f = open(dbfile, 'r')
    text = f.read()
    f.close()
    db.write(';'.join(text.split(' ')))

db.close()

In [218]:
parser = Bio.PDB.PDBParser()
ppb = Bio.PDB.CaPPBuilder()

query_path = "../sandbox/query.fasta"
query_file = open(query_path, 'w')

pdbfiles = glob.glob("../data/structures/*.pdb")
for pdb in pdbfiles:
    structure = parser.get_structure(id=pdb[-8:-4], file=pdb)
    for chain in structure[0]:
        #print(structure.id)
        try:
            query_file.write('>%s;%s\n%s\n' % (structure.id, chain.id, ''.join([str(s.get_sequence()) for s in ppb.build_peptides(chain)])))
        except Exception:
            continue
        
query_file.close()

'parser = Bio.PDB.PDBParser()\nppb = Bio.PDB.CaPPBuilder()\n\nquery_path = "../sandbox/query.fasta"\nquery_file = open(query_path, \'w\')\n\npdbfiles = glob.glob("../data/structures/*.pdb")\nfor pdb in pdbfiles:\n    structure = parser.get_structure(id=pdb[-8:-4], file=pdb)\n    for chain in structure[0]:\n        #print(structure.id)\n        try:\n            query_file.write(\'>%s;%s\n%s\n\' % (structure.id, chain.id, \'\'.join([str(s.get_sequence()) for s in ppb.build_peptides(chain)])))\n        except Exception:\n            continue\n        \nquery_file.close()'

In [219]:
subprocess.call(["blastp", "-query", query_path, 
                 "-subject", mhc_db_path, "-out", 
                 blast_output_path, "-outfmt", str(6), "-num_alignments", str(1)])

0

In [227]:
epitope_db = pd.read_csv(epitope_db_path, header=None, skiprows=1, low_memory=False)

In [232]:
epitope_db[epitope_db[2].str.contains('AAAAAAAAAFAAK', regex=True)]

,0,1,2,3,4,5,6,7,8,9,10,11
4,5,Linear peptide,AAAAAAAAAFAAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
